In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from db import helper as db_helper
from db.models import SpotRate
import pandas as pd

In [2]:
session = db_helper.start_session()

df = pd.read_sql_table('spot_rates', session.bind)

2019-05-02 22:26:15,651 INFO sqlalchemy.engine.base.Engine select version()
2019-05-02 22:26:15,653 INFO sqlalchemy.engine.base.Engine {}
2019-05-02 22:26:15,660 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-05-02 22:26:15,661 INFO sqlalchemy.engine.base.Engine {}
2019-05-02 22:26:15,666 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-02 22:26:15,667 INFO sqlalchemy.engine.base.Engine {}
2019-05-02 22:26:15,671 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-02 22:26:15,672 INFO sqlalchemy.engine.base.Engine {}
2019-05-02 22:26:15,673 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-05-02 22:26:15,674 INFO sqlalchemy.engine.base.Engine {}
2019-05-02 22:26:15,677 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind = 'r'
2019-05-02 22:

/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


2019-05-02 22:26:15,727 INFO sqlalchemy.engine.base.Engine 
                SELECT a.attname
                FROM pg_attribute a JOIN (
                    SELECT unnest(ix.indkey) attnum,
                           generate_subscripts(ix.indkey, 1) ord
                    FROM pg_index ix
                    WHERE ix.indrelid = %(table_oid)s AND ix.indisprimary
                    ) k ON a.attnum=k.attnum
                WHERE a.attrelid = %(table_oid)s
                ORDER BY k.ord
            
2019-05-02 22:26:15,730 INFO sqlalchemy.engine.base.Engine {'table_oid': 16656}
2019-05-02 22:26:15,735 INFO sqlalchemy.engine.base.Engine 
        SELECT conname
           FROM  pg_catalog.pg_constraint r
           WHERE r.conrelid = %(table_oid)s AND r.contype = 'p'
           ORDER BY 1
        
2019-05-02 22:26:15,736 INFO sqlalchemy.engine.base.Engine {'table_oid': 16656}
2019-05-02 22:26:15,743 INFO sqlalchemy.engine.base.Engine 
          SELECT r.conname,
                pg_catalog.

In [4]:
# remove unnecessary columns
df.drop(columns=['id', 'url', 'created_at', 'updated_at'], inplace=True)
df.head()

,day,month,year,base_currency,target_currency,base_value,target_spot_rate,target_52wk_high,target_52wk_low
0,31,5,2007,USD,Australian Dollar,1.0,1.2082,NaN,NaN
1,31,5,2007,USD,Canadian Dollar,1.0,1.0691,NaN,NaN
2,31,5,2007,USD,Chinese Yuan,1.0,7.6450,NaN,NaN
3,31,5,2007,USD,Cyprus Pound,1.0,0.4336,NaN,NaN
4,31,5,2007,USD,Czech Koruna,1.0,21.0323,NaN,NaN


In [5]:
# Quick snapshot of the data
print(df.describe())

# lack of significant data in the 52wk columns
df.drop(columns=['target_52wk_high', 'target_52wk_low', 'base_value'], inplace=True)
df.head()

                 day          month           year  base_value  \
count  289056.000000  289056.000000  289056.000000    289056.0   
mean       15.668158       6.569263    2009.311369         1.0   
std         8.705023       3.438268       5.233624         0.0   
min         1.000000       1.000000    2000.000000         1.0   
25%         8.000000       4.000000    2005.000000         1.0   
50%        16.000000       7.000000    2009.000000         1.0   
75%        23.000000      10.000000    2014.000000         1.0   
max        31.000000      12.000000    2018.000000         1.0   

       target_spot_rate  target_52wk_high  target_52wk_low  
count     289056.000000         60.000000        60.000000  
mean          77.180395         81.021365        73.688215  
std          278.083068        250.473364       230.362989  
min            0.000000          0.798300         0.697900  
25%            1.973000          1.998150         1.795775  
50%            6.719200          7.6589

,day,month,year,base_currency,target_currency,target_spot_rate
0,31,5,2007,USD,Australian Dollar,1.2082
1,31,5,2007,USD,Canadian Dollar,1.0691
2,31,5,2007,USD,Chinese Yuan,7.6450
3,31,5,2007,USD,Cyprus Pound,0.4336
4,31,5,2007,USD,Czech Koruna,21.0323


In [6]:
# create Date
import datetime
df['date'] = df.apply(lambda x: datetime.date(x['year'], x['month'], x['day']), axis=1)
df['date'] = pd.to_datetime(df['date'])
df['week'] = df['date'].dt.week

df.head()

,day,month,year,base_currency,target_currency,target_spot_rate,date,week
0,31,5,2007,USD,Australian Dollar,1.2082,2007-05-31,22
1,31,5,2007,USD,Canadian Dollar,1.0691,2007-05-31,22
2,31,5,2007,USD,Chinese Yuan,7.6450,2007-05-31,22
3,31,5,2007,USD,Cyprus Pound,0.4336,2007-05-31,22
4,31,5,2007,USD,Czech Koruna,21.0323,2007-05-31,22


In [7]:
# Convert target_curency into Iso Symbol
currency_dict = {'Japanese Yen': 'JPY','Malaysian ringgit': 'MYR','New Zealand Dollar': 'NZD','Norwegian Krone': 'NOK','Polish Zloty': 'PLN','Russian Ruble': 'RUB','Saudi Riyal': 'SAR','Singapore Dollar': 'SGD','South African Rand': 'ZAR','South Korean Won': 'KRW','Swedish Krona': 'SEK','Swiss Franc': 'CHF','Taiwan Dollar': 'TWD','Thai Baht': 'THB','Turkish Lira': 'TRY','US Dollar': 'USD','Australian Dollar': 'AUD','Canadian Dollar': 'CAD','Chinese Yuan': 'Yuan','Cyprus Pound': 'CYP','Czech Koruna': 'CZK','Danish Krone': 'DKK','Estonian Kroon': 'EEK','Euro': 'EUR','Hong Kong Dollar': 'HKD','Hungarian Forint': 'HUF','Indian Rupee': 'INR','Israeli Shekel': 'ILS','Latvian Lats': 'LVL','Lithuanian Litas': 'LTL','Maltese Lira': 'MTL','Slovak Koruna': 'SKK','Slovenian Tolar': 'SIT','Sterling': 'GBP','Swedish Krona ': 'SEK','Brazilian Real': 'BRL','Austrian Schilling': 'ATS','Belgian Franc': 'BEF','Deutschemark': 'DEM','Finnish Markka': 'FIM','French Franc': 'FRF','Greek Drachma': 'GRD','Irish Punt': 'IEP','Italian Lire': 'ITL','Netherlands Guilder': 'NLG','Portuguese Escudo': 'PTE','Spanish Peseta': 'ESP'}

df['target_currency'].replace('Latvian Lats\r\nCurrency joined the Euro on 01/01/2014', 'Latvian Lats', inplace=True)
df['target_currency_symbol'] = df['target_currency'].apply(lambda x: currency_dict[x])
df.drop(columns=['target_currency'], inplace=True)
df.head()

,day,month,year,base_currency,target_spot_rate,date,week,target_currency_symbol
0,31,5,2007,USD,1.2082,2007-05-31,22,AUD
1,31,5,2007,USD,1.0691,2007-05-31,22,CAD
2,31,5,2007,USD,7.6450,2007-05-31,22,Yuan
3,31,5,2007,USD,0.4336,2007-05-31,22,CYP
4,31,5,2007,USD,21.0323,2007-05-31,22,CZK


In [8]:
# Reorder columns
df = df[['date', 'year', 'month', 'week', 'day', 'base_currency', 'target_currency_symbol', 'target_spot_rate']]
df.head()

,date,year,month,week,day,base_currency,target_currency_symbol,target_spot_rate
0,2007-05-31,2007,5,22,31,USD,AUD,1.2082
1,2007-05-31,2007,5,22,31,USD,CAD,1.0691
2,2007-05-31,2007,5,22,31,USD,Yuan,7.6450
3,2007-05-31,2007,5,22,31,USD,CYP,0.4336
4,2007-05-31,2007,5,22,31,USD,CZK,21.0323


In [9]:
# Split DF by currency
gbp_df = df[(df['base_currency']=='GBP')]
usd_df = df[(df['base_currency']=='USD')]
eur_df = df[(df['base_currency']=='EUR')]

In [10]:
import seaborn as sns
from matplotlib import pyplot as plt

plt.figure(figsize=(16,8))
plt.style.use('seaborn-whitegrid')

gbp_usd = gbp_df[gbp_df['target_currency_symbol']=='USD']
gbp_eur = gbp_df[gbp_df['target_currency_symbol']=='EUR']
gbp_jpy = gbp_df[gbp_df['target_currency_symbol']=='JPY']

ax = sns.lineplot(x='date', y='target_spot_rate', data=gbp_usd, label='1 GBP = USD')
ax = sns.lineplot(x='date', y='target_spot_rate', data=gbp_eur, label='1 GBP = EUR')
# ax = sns.lineplot(x='date', y='target_spot_rate', data=gbp_jpy, label='1 GBP = JPY')

plt.show()

<Figure size 1600x800 with 1 Axes>

In [10]:
# %%timeit
# def change_from_n_days_ago(df, target_currency_symbol, current_date, current_rate, n):

#     df = df[df['target_currency_symbol']==target_currency_symbol]

#     try:
#         previous_period_rate = df[df.date < current_date - pd.to_timedelta(n, 'D')].sort_values('date', ascending=False).iloc[0]['target_spot_rate']
#         change = current_rate - previous_period_rate
#     except:
#         change = None

#     return change

# temp = gbp_df.head(10)
# temp['test'] = temp.apply(lambda x: change_from_n_days_ago(temp, x['target_currency_symbol'], x['date'], x['target_spot_rate'], 1), axis=1)
# temp.sort_values(['target_currency_symbol', 'date'], ascending=False)

In [11]:
gbp_df.shape

(122968, 8)

In [12]:
gbp_pivot_df = gbp_df.pivot(index='date', columns='target_currency_symbol', values='target_spot_rate')

In [13]:
# Feature Generation

In [14]:
# get relative change in prices of currencies
gbp_pivot_d_change = gbp_pivot_df/gbp_pivot_df.shift(1, freq=pd.to_timedelta(1, 'D')) - 1
gbp_pivot_d_change = gbp_pivot_d_change.unstack().reset_index().rename({0: '1d_return'}, axis=1)

gbp_pivot_w_change = gbp_pivot_df/gbp_pivot_df.shift(7, freq=pd.to_timedelta(1, 'D')) - 1
gbp_pivot_w_change = gbp_pivot_w_change.unstack().reset_index().rename({0: '1w_return'}, axis=1)

gbp_pivot_m_change = gbp_pivot_df/gbp_pivot_df.shift(30, freq=pd.to_timedelta(1, 'D')) - 1
gbp_pivot_m_change = gbp_pivot_m_change.unstack().reset_index().rename({0: '1m_return'}, axis=1)

gbp_pivot_y_change = gbp_pivot_df/gbp_pivot_df.shift(365, freq=pd.to_timedelta(1, 'D')) - 1
gbp_pivot_y_change = gbp_pivot_y_change.unstack().reset_index().rename({0: '1y_return'}, axis=1)

In [15]:
gbp_df = gbp_df.merge(gbp_pivot_d_change, how='left', on=['date', 'target_currency_symbol'])
gbp_df = gbp_df.merge(gbp_pivot_w_change, how='left', on=['date', 'target_currency_symbol'])
gbp_df = gbp_df.merge(gbp_pivot_m_change, how='left', on=['date', 'target_currency_symbol'])
gbp_df = gbp_df.merge(gbp_pivot_y_change, how='left', on=['date', 'target_currency_symbol'])
gbp_df = gbp_df[~gbp_df['base_currency'].isnull()]
gbp_df.head()

,date,year,month,week,day,base_currency,target_currency_symbol,target_spot_rate,1d_return,1w_return,1m_return,1y_return
0,2018-10-01,2018,10,40,1,GBP,AUD,1.8068,NaN,-0.000940,NaN,NaN
1,2018-10-01,2018,10,40,1,GBP,CAD,1.6729,NaN,-0.015420,NaN,NaN
2,2018-10-01,2018,10,40,1,GBP,Yuan,8.9579,NaN,-0.005562,NaN,NaN
3,2018-10-01,2018,10,40,1,GBP,CZK,29.0475,NaN,0.017208,NaN,NaN
4,2018-10-01,2018,10,40,1,GBP,DKK,8.3960,NaN,0.009875,NaN,NaN


In [16]:
# get volatility of prices
week_vol = gbp_df.groupby(['week', 'month', 'year', 'target_currency_symbol'])['target_spot_rate'].std()
week_vol = week_vol.reset_index().rename({'target_spot_rate': '1w_vol'}, axis=1)

month_vol = gbp_df.groupby(['month', 'year', 'target_currency_symbol'])['target_spot_rate'].std()
month_vol = month_vol.reset_index().rename({'target_spot_rate': '1m_vol'}, axis=1)

year_vol = gbp_df.groupby(['year', 'target_currency_symbol'])['target_spot_rate'].std()
year_vol = year_vol.reset_index().rename({'target_spot_rate': '1y_vol'}, axis=1)

week_vol.head()
month_vol.head()
year_vol.head()

,year,target_currency_symbol,1y_vol
0,2000,ATS,0.504681
1,2000,AUD,0.079927
2,2000,BEF,1.479057
3,2000,CAD,0.069270
4,2000,CHF,0.063835


In [17]:
gbp_df = gbp_df.merge(week_vol, how='left', on=['week', 'month', 'year', 'target_currency_symbol'])
gbp_df = gbp_df.merge(month_vol, how='left', on=['month', 'year', 'target_currency_symbol'])
gbp_df = gbp_df.merge(year_vol, how='left', on=['year', 'target_currency_symbol'])

gbp_df.head()

,date,year,month,week,day,base_currency,target_currency_symbol,target_spot_rate,1d_return,1w_return,1m_return,1y_return,1w_vol,1m_vol,1y_vol
0,2018-10-01,2018,10,40,1,GBP,AUD,1.8068,NaN,-0.000940,NaN,NaN,0.020635,0.020166,0.032744
1,2018-10-01,2018,10,40,1,GBP,CAD,1.6729,NaN,-0.015420,NaN,NaN,0.010306,0.017219,0.040853
2,2018-10-01,2018,10,40,1,GBP,Yuan,8.9579,NaN,-0.005562,NaN,NaN,0.024415,0.084348,0.147415
3,2018-10-01,2018,10,40,1,GBP,CZK,29.0475,NaN,0.017208,NaN,NaN,0.125021,0.182459,0.336169
4,2018-10-01,2018,10,40,1,GBP,DKK,8.3960,NaN,0.009875,NaN,NaN,0.037457,0.051445,0.081594


In [18]:
# reshape dataframe to use returns from other currencies as features.
cols = ['target_spot_rate', '1d_return', '1w_return', '1m_return', '1y_return', '1w_vol', '1m_vol', '1y_vol']

gbp_df['target_currency_symbol'].unique()

currencies = ['AUD', 'CAD', 'USD', 'EUR', 'JPY']
# currencies = gbp_df['target_currency_symbol'].unique()

test_df = gbp_df

for cur in currencies:
    temp_df = gbp_df[gbp_df['target_currency_symbol']==cur][['date']+cols]
    temp_df = temp_df.reset_index().rename({col: cur+'_'+col for col in cols}, axis=1)
    temp_df.drop(columns=['index'], inplace=True, axis=1)

    test_df = test_df.merge(temp_df, how='left', on=['date'])


print(test_df.shape)
test_df.drop(columns=['target_currency_symbol']+cols, axis=1, inplace=True)
test_df.drop_duplicates(inplace=True)
print(test_df.shape)
test_df.reset_index().drop('index', inplace=True, axis=1)
test_df.head()

(122968, 55)
(4811, 46)


,date,year,month,week,day,base_currency,AUD_target_spot_rate,AUD_1d_return,AUD_1w_return,AUD_1m_return,...,EUR_1m_vol,EUR_1y_vol,JPY_target_spot_rate,JPY_1d_return,JPY_1w_return,JPY_1m_return,JPY_1y_return,JPY_1w_vol,JPY_1m_vol,JPY_1y_vol
0,2018-10-01,2018,10,40,1,GBP,1.8068,NaN,-0.000940,NaN,...,0.006837,0.011365,148.6250,NaN,0.005131,NaN,NaN,0.471788,1.841819,3.361116
26,2016-08-18,2016,8,33,18,GBP,1.7147,0.006338,0.020533,-0.022294,...,0.011928,0.066346,131.8043,0.010551,0.003097,-0.056883,-0.321947,0.683814,1.655313,14.073918
52,2018-06-12,2018,6,24,12,GBP,1.7538,-0.003296,-0.000513,NaN,...,0.004213,0.011365,147.2726,0.000493,0.005108,NaN,0.058184,0.301753,0.956713,3.361116
78,2012-04-24,2012,4,17,24,GBP,1.5670,0.001982,0.023180,NaN,...,0.008094,0.025680,131.0572,0.004344,0.018836,NaN,NaN,0.291318,1.259400,4.314676
104,2015-09-16,2015,9,38,16,GBP,2.1577,-0.003970,-0.013127,0.020817,...,0.008889,0.033250,186.9523,0.013213,0.005708,-0.036352,0.075523,1.426208,2.007801,5.283531


In [19]:
test_df.describe()

,year,month,week,day,AUD_target_spot_rate,AUD_1d_return,AUD_1w_return,AUD_1m_return,AUD_1y_return,AUD_1w_vol,...,EUR_1m_vol,EUR_1y_vol,JPY_target_spot_rate,JPY_1d_return,JPY_1w_return,JPY_1m_return,JPY_1y_return,JPY_1w_vol,JPY_1m_vol,JPY_1y_vol
count,4811.00000,4811.000000,4811.00000,4811.000000,4797.000000,3794.000000,4661.000000,2798.000000,3609.000000,4729.000000,...,4797.000000,4797.000000,4797.000000,3794.000000,4661.000000,2798.000000,3609.000000,4729.000000,4797.000000,4797.000000
mean,2008.96799,6.519850,26.54417,15.673872,2.111388,0.000057,-0.000112,-0.000850,-0.015768,0.012387,...,0.011181,0.030376,170.750455,0.000050,-0.000042,-0.000163,0.003957,1.055759,2.139659,6.950896
std,5.48487,3.436591,14.97604,8.708985,0.422968,0.007214,0.015534,0.032249,0.100358,0.009150,...,0.006297,0.013293,31.014276,0.008029,0.018052,0.036493,0.130945,0.785514,1.331464,5.083983
min,2000.00000,1.000000,1.00000,1.000000,1.442800,-0.069132,-0.090420,-0.123681,-0.323618,0.000212,...,0.003656,0.011365,117.074900,-0.111059,-0.170378,-0.248708,-0.425379,0.003253,0.610253,3.361116
25%,2004.00000,4.000000,13.00000,8.000000,1.730900,-0.004154,-0.008974,-0.021397,-0.084154,0.006632,...,0.006837,0.022052,144.447400,-0.003896,-0.009304,-0.020624,-0.060113,0.589164,1.291389,4.093324
50%,2009.00000,7.000000,27.00000,16.000000,2.075700,-0.000101,-0.000619,-0.003456,-0.011536,0.010181,...,0.009445,0.028038,171.472000,0.000301,0.000460,0.002089,0.030744,0.879883,1.818837,5.336761
75%,2014.00000,10.000000,40.00000,23.000000,2.470400,0.003860,0.008703,0.016814,0.053139,0.015259,...,0.013869,0.034622,194.059200,0.004366,0.010198,0.020984,0.088268,1.315852,2.610066,7.786192
max,2018.00000,12.000000,53.00000,31.000000,3.028000,0.061400,0.161188,0.209706,0.282853,0.086130,...,0.052491,0.066346,250.108300,0.064945,0.086502,0.133297,0.277856,10.626668,12.424410,25.694912


In [23]:
test_df.drop('base_currency', axis=1, inplace=True)
test_df.drop('date', axis=1, inplace=True)
test_df.dtypes

year                      int64
month                     int64
week                      int64
day                       int64
AUD_target_spot_rate    float64
AUD_1d_return           float64
AUD_1w_return           float64
AUD_1m_return           float64
AUD_1y_return           float64
AUD_1w_vol              float64
AUD_1m_vol              float64
AUD_1y_vol              float64
CAD_target_spot_rate    float64
CAD_1d_return           float64
CAD_1w_return           float64
CAD_1m_return           float64
CAD_1y_return           float64
CAD_1w_vol              float64
CAD_1m_vol              float64
CAD_1y_vol              float64
USD_target_spot_rate    float64
USD_1d_return           float64
USD_1w_return           float64
USD_1m_return           float64
USD_1y_return           float64
USD_1w_vol              float64
USD_1m_vol              float64
USD_1y_vol              float64
EUR_target_spot_rate    float64
EUR_1d_return           float64
EUR_1w_return           float64
EUR_1m_r

In [25]:
import numpy as np
# Create Model
print(test_df.columns)

for target_col in ['AUD_target_spot_rate','AUD_1d_return','AUD_1w_return','AUD_1m_return','AUD_1y_return','AUD_1w_vol','AUD_1m_vol','AUD_1y_vol','CAD_target_spot_rate','CAD_1d_return','CAD_1w_return','CAD_1m_return','CAD_1y_return','CAD_1w_vol','CAD_1m_vol','CAD_1y_vol','USD_target_spot_rate','USD_1d_return','USD_1w_return','USD_1m_return','USD_1y_return','USD_1w_vol','USD_1m_vol','USD_1y_vol','EUR_target_spot_rate','EUR_1d_return','EUR_1w_return','EUR_1m_return','EUR_1y_return','EUR_1w_vol','EUR_1m_vol','EUR_1y_vol','JPY_target_spot_rate','JPY_1d_return','JPY_1w_return','JPY_1m_return','JPY_1y_return','JPY_1w_vol','JPY_1m_vol','JPY_1y_vol']:
    print(target_col)

# target_col = 'USD_1y_return'

    print(test_df.shape)
    test_df_full = test_df.dropna()
    print(test_df_full.shape)

    y = test_df_full[target_col].values
    X = test_df_full.drop(columns=target_col).values

#     from sklearn.model_selection import train_test_split
#     from tpot import TPOTClassifier, TPOTRegressor

#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=None)

#     tpot = TPOTRegressor(verbosity=2, max_time_mins=5, population_size=10, generations=5)
#     tpot.fit(X_train, y_train)
#     print(tpot.score(X_test, y_test))

Index(['year', 'month', 'week', 'day', 'AUD_target_spot_rate', 'AUD_1d_return',
       'AUD_1w_return', 'AUD_1m_return', 'AUD_1y_return', 'AUD_1w_vol',
       'AUD_1m_vol', 'AUD_1y_vol', 'CAD_target_spot_rate', 'CAD_1d_return',
       'CAD_1w_return', 'CAD_1m_return', 'CAD_1y_return', 'CAD_1w_vol',
       'CAD_1m_vol', 'CAD_1y_vol', 'USD_target_spot_rate', 'USD_1d_return',
       'USD_1w_return', 'USD_1m_return', 'USD_1y_return', 'USD_1w_vol',
       'USD_1m_vol', 'USD_1y_vol', 'EUR_target_spot_rate', 'EUR_1d_return',
       'EUR_1w_return', 'EUR_1m_return', 'EUR_1y_return', 'EUR_1w_vol',
       'EUR_1m_vol', 'EUR_1y_vol', 'JPY_target_spot_rate', 'JPY_1d_return',
       'JPY_1w_return', 'JPY_1m_return', 'JPY_1y_return', 'JPY_1w_vol',
       'JPY_1m_vol', 'JPY_1y_vol'],
      dtype='object')
AUD_target_spot_rate
(4811, 44)
(2584, 44)
{-0.4253786177022306: 1, -0.41899314632738427: 1, -0.4084482579616011: 1, -0.4061410903895599: 1, -0.40549726555485843: 1, -0.40421044892579105: 1, -0.4029

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
# Create Model
print(test_df.columns)

from imblearn.over_sampling import RandomOverSampler

# for target_col in ['AUD_target_spot_rate','AUD_1d_return','AUD_1w_return','AUD_1m_return','AUD_1y_return','AUD_1w_vol','AUD_1m_vol','AUD_1y_vol','CAD_target_spot_rate','CAD_1d_return','CAD_1w_return','CAD_1m_return','CAD_1y_return','CAD_1w_vol','CAD_1m_vol','CAD_1y_vol','USD_target_spot_rate','USD_1d_return','USD_1w_return','USD_1m_return','USD_1y_return','USD_1w_vol','USD_1m_vol','USD_1y_vol','EUR_target_spot_rate','EUR_1d_return','EUR_1w_return','EUR_1m_return','EUR_1y_return','EUR_1w_vol','EUR_1m_vol','EUR_1y_vol','JPY_target_spot_rate','JPY_1d_return','JPY_1w_return','JPY_1m_return','JPY_1y_return','JPY_1w_vol','JPY_1m_vol','JPY_1y_vol']:
#     print(target_col)

for target_col in ['AUD_1d_return','AUD_1w_return','AUD_1m_return','AUD_1y_return'][2:]:
    print(target_col)
    
    print(test_df.shape)
    test_df_full = test_df.dropna()
    print(test_df_full.shape)

    y = test_df_full[target_col].values
    y = [1 if _ < 0 else 0 for _ in y]
        
    X_df = test_df_full.drop(columns=target_col)   
    X = X_df.values
    
    unique, counts = np.unique(y, return_counts=True)
    print(dict(zip(unique, counts)))
    ros = RandomOverSampler(random_state=0)
    X, y = ros.fit_resample(X, y)
    unique, counts = np.unique(y, return_counts=True)
    print(dict(zip(unique, counts)))

    from sklearn.model_selection import train_test_split
    from tpot import TPOTClassifier, TPOTRegressor

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=None)
    
    tpot = TPOTClassifier(verbosity=2, max_time_mins=5, population_size=10, generations=5)
    tpot.fit(X_train, y_train)
    print(tpot.score(X_test, y_test))
    
    break

Index(['year', 'month', 'week', 'day', 'AUD_target_spot_rate', 'AUD_1d_return',
       'AUD_1w_return', 'AUD_1m_return', 'AUD_1y_return', 'AUD_1w_vol',
       'AUD_1m_vol', 'AUD_1y_vol', 'CAD_target_spot_rate', 'CAD_1d_return',
       'CAD_1w_return', 'CAD_1m_return', 'CAD_1y_return', 'CAD_1w_vol',
       'CAD_1m_vol', 'CAD_1y_vol', 'USD_target_spot_rate', 'USD_1d_return',
       'USD_1w_return', 'USD_1m_return', 'USD_1y_return', 'USD_1w_vol',
       'USD_1m_vol', 'USD_1y_vol', 'EUR_target_spot_rate', 'EUR_1d_return',
       'EUR_1w_return', 'EUR_1m_return', 'EUR_1y_return', 'EUR_1w_vol',
       'EUR_1m_vol', 'EUR_1y_vol', 'JPY_target_spot_rate', 'JPY_1d_return',
       'JPY_1w_return', 'JPY_1m_return', 'JPY_1y_return', 'JPY_1w_vol',
       'JPY_1m_vol', 'JPY_1y_vol'],
      dtype='object')
AUD_1m_return
(4811, 44)
(2584, 44)
{0: 1159, 1: 1425}
{0: 1425, 1: 1425}


HBox(children=(IntProgress(value=0, description='Optimization Progress', max=10), HTML(value='')))

Generation 1 - Current best internal CV score: 0.8848792926087242


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=4, max_features=0.55, min_samples_leaf=8, min_samples_split=9, n_estimators=100, subsample=0.6000000000000001)
0.876577840112202


In [36]:
test_df_full.columns

Index(['year', 'month', 'week', 'day', 'AUD_target_spot_rate', 'AUD_1d_return',
       'AUD_1w_return', 'AUD_1m_return', 'AUD_1y_return', 'AUD_1w_vol',
       'AUD_1m_vol', 'AUD_1y_vol', 'CAD_target_spot_rate', 'CAD_1d_return',
       'CAD_1w_return', 'CAD_1m_return', 'CAD_1y_return', 'CAD_1w_vol',
       'CAD_1m_vol', 'CAD_1y_vol', 'USD_target_spot_rate', 'USD_1d_return',
       'USD_1w_return', 'USD_1m_return', 'USD_1y_return', 'USD_1w_vol',
       'USD_1m_vol', 'USD_1y_vol', 'EUR_target_spot_rate', 'EUR_1d_return',
       'EUR_1w_return', 'EUR_1m_return', 'EUR_1y_return', 'EUR_1w_vol',
       'EUR_1m_vol', 'EUR_1y_vol', 'JPY_target_spot_rate', 'JPY_1d_return',
       'JPY_1w_return', 'JPY_1m_return', 'JPY_1y_return', 'JPY_1w_vol',
       'JPY_1m_vol', 'JPY_1y_vol'],
      dtype='object')

In [40]:
def show_feature_importance(model, X, y, cols):
    """
    Random sample X, y
    """
    from sklearn.model_selection import cross_val_score
    n_feats = X.shape[1]

    feature_importance = {}

    for i in range(n_feats):
        print(f'{i} of {n_feats} predicted.')
        x = X.values[:, i].reshape(-1, 1)
        scores = cross_val_score(model, x, y, cv=3)
        feature_importance[cols[i]] = scores.mean()
    
    pprint(feature_importance)

    return feature_importance

from pprint import pprint
show_feature_importance(tpot.fitted_pipeline_, pd.DataFrame(X), pd.DataFrame(y), X_df.columns)

0 of 43 predicted.


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

1 of 43 predicted.


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2 of 43 predicted.


...

42 of 43 predicted.


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'AUD_1d_return': 0.5428070175438596,
 'AUD_1m_vol': 0.7582456140350877,
 'AUD_1w_return': 0.6589473684210526,
 'AUD_1w_vol': 0.6442105263157895,
 'AUD_1y_return': 0.6028070175438597,
 'AUD_1y_vol': 0.5673684210526315,
 'AUD_target_spot_rate': 0.5975438596491228,
 'CAD_1d_return': 0.5270175438596492,
 'CAD_1m_return': 0.7196491228070175,
 'CAD_1m_vol': 0.7592982456140351,
 'CAD_1w_return': 0.5659649122807018,
 'CAD_1w_vol': 0.643859649122807,
 'CAD_1y_return': 0.5543859649122808,
 'CAD_1y_vol': 0.5708771929824561,
 'CAD_target_spot_rate': 0.5743859649122808,
 'EUR_1d_return': 0.5480701754385965,
 'EUR_1m_return': 0.6298245614035087,
 'EUR_1m_vol': 0.7463157894736843,
 'EUR_1w_return': 0.5336842105263158,
 'EUR_1w_vol': 0.6575438596491229,
 'EUR_1y_return': 0.5673684210526315,
 'EUR_1y_vol': 0.5628070175438596,
 'EUR_target_spot_rate': 0.5757894736842105,
 'JPY_1d_return': 0.5214035087719299,
 'JPY_1m_return': 0.6021052631578948,
 'JPY_1m_vol': 0.7298245614035088,
 'JPY_1w_return': 0.52

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'AUD_1d_return': 0.5428070175438596,
 'AUD_1m_vol': 0.7582456140350877,
 'AUD_1w_return': 0.6589473684210526,
 'AUD_1w_vol': 0.6442105263157895,
 'AUD_1y_return': 0.6028070175438597,
 'AUD_1y_vol': 0.5673684210526315,
 'AUD_target_spot_rate': 0.5975438596491228,
 'CAD_1d_return': 0.5270175438596492,
 'CAD_1m_return': 0.7196491228070175,
 'CAD_1m_vol': 0.7592982456140351,
 'CAD_1w_return': 0.5659649122807018,
 'CAD_1w_vol': 0.643859649122807,
 'CAD_1y_return': 0.5543859649122808,
 'CAD_1y_vol': 0.5708771929824561,
 'CAD_target_spot_rate': 0.5743859649122808,
 'EUR_1d_return': 0.5480701754385965,
 'EUR_1m_return': 0.6298245614035087,
 'EUR_1m_vol': 0.7463157894736843,
 'EUR_1w_return': 0.5336842105263158,
 'EUR_1w_vol': 0.6575438596491229,
 'EUR_1y_return': 0.5673684210526315,
 'EUR_1y_vol': 0.5628070175438596,
 'EUR_target_spot_rate': 0.5757894736842105,
 'JPY_1d_return': 0.5214035087719299,
 'JPY_1m_return': 0.6021052631578948,
 'JPY_1m_vol': 0.7298245614035088,
 'JPY_1w_return': 0.52

In [ ]:
# import numpy as np
# temp = gbp_df[gbp_df['target_currency_symbol']=='USD'].sort_values('date', ascending=False)
# temp.date = temp.date.values.astype(np.int64)
# temp.dropna(inplace=True)

# print(temp.head(10))

# vol = ['1w_vol', '1m_vol', '1y_vol']

# # X = temp[['target_spot_rate', 'date', '1w_return', '1m_return', '1y_return']]
# # y = temp['1d_return']

# # X = temp[['target_spot_rate', 'date', '1w_return', '1m_return', '1y_return'] + vol]
# # y = temp['1d_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1m_return', '1y_return']]
# # y = temp['1w_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1m_return', '1y_return'] + vol]
# # y = temp['1w_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1y_return']]
# # y = temp['1m_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1y_return'] + vol]
# # y = temp['1m_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1m_return']]
# # y = temp['1y_return']

# X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1m_return'] + vol]
# y = temp['1y_return']

# import statsmodels.api as sm

# # Note the difference in argument order
# model = sm.OLS(y, X).fit()
# predictions = model.predict(X) # make the predictions by the model

# # Print out the statistics
# model.summary()

In [ ]:
# # X = temp[['target_spot_rate', 'date', '1w_return', '1m_return', '1y_return']]
# # y = temp['1d_return']

# # X = temp[['target_spot_rate', 'date', '1w_return', '1m_return', '1y_return'] + vol]
# # y = temp['1d_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1m_return', '1y_return']]
# # y = temp['1w_return']

# X = temp[['target_spot_rate', 'date', '1d_return', '1m_return', '1y_return'] + vol]
# y = temp['1w_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1y_return']]
# # y = temp['1m_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1y_return'] + vol]
# # y = temp['1m_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1m_return']]
# # y = temp['1y_return']

# # X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1m_return'] + vol]
# # y = temp['1y_return']

In [ ]:
# import warnings
# from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
# warnings.filterwarnings(action='ignore', category=DataConversionWarning)
# warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
# warnings.filterwarnings(action='always', category=ConvergenceWarning)

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LassoCV
# from sklearn.cross_validation import train_test_split
# from sklearn.pipeline import make_pipeline

# lasso_eps = 0.0001
# lasso_nalpha=20
# lasso_iter=1000

# X = temp[['target_spot_rate', 'date', '1w_return', '1m_return', '1y_return'] + vol]
# y = temp['1d_return']

# X = temp[['target_spot_rate', 'date', '1d_return', '1m_return', '1y_return'] + vol]
# y = temp['1w_return']

# X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1y_return'] + vol]
# y = temp['1m_return']

# X = temp[['target_spot_rate', 'date', '1d_return', '1w_return', '1m_return'] + vol]
# y = temp['1y_return']

# # Min and max degree of polynomials features to consider
# degree_min = 2
# degree_max = 6

# # Test/train split
# X_train, X_test, y_train, y_test = train_test_split(X, y)

# # Make a pipeline model with polynomial transformation and LASSO regression with cross-validation, run it for increasing degree of polynomial (complexity of the model)
# for degree in range(degree_min,degree_max+1):
#     model = make_pipeline(PolynomialFeatures(degree, interaction_only=False), LassoCV(eps=lasso_eps,n_alphas=lasso_nalpha,max_iter=lasso_iter,
# normalize=True,cv=5))
#     model.fit(X_train,y_train)
#     test_pred = np.array(model.predict(X_test))
#     RMSE=np.sqrt(np.sum(np.square(test_pred-y_test)))
#     test_score = model.score(X_test,y_test)
#     print(degree, RMSE, test_score)

In [ ]:
# # Try other currencies

# # cols=['Japanese Yen','Malaysian ringgit','New Zealand Dollar','Norwegian Krone','Russian Ruble','Saudi Riyal','Singapore Dollar','South African Rand','South Korean Won','Swedish Krona','Taiwan Dollar','Thai Baht','Turkish Lira','US Dollar','Australian Dollar','Canadian Dollar','Chinese Yuan','Euro','Hong Kong Dollar','Indian Rupee','Finnish Markka']
# cols=['Japanese Yen','US Dollar','Euro','Hong Kong Dollar']
# vals=['target_spot_rate','1d_return','1w_return','1m_return','1y_return','1w_vol','1m_vol','1y_vol']

# test_df = gbp_df[gbp_df['target_currency'].isin(cols)]
# test_df = test_df.pivot(index='date', columns='target_currency', values=vals)
# test_df.dropna(inplace=True)

# y_currency = 'US Dollar'
# y_value = '1w_return'
# y_string = y_value + '_' + y_currency

# test_df.columns = test_df.columns.map('_'.join)
# X = test_df.drop(y_string, axis=1)
# y = test_df[y_string]

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LassoCV
# from sklearn.cross_validation import train_test_split
# from sklearn.pipeline import make_pipeline

# lasso_eps = 0.0001
# lasso_nalpha=20
# lasso_iter=5000

# # Min and max degree of polynomials features to consider
# degree_min = 2
# degree_max = 4

# # Test/train split
# X_train, X_test, y_train, y_test = train_test_split(X, y)

# # Make a pipeline model with polynomial transformation and LASSO regression with cross-validation, run it for increasing degree of polynomial (complexity of the model)
# for degree in range(degree_min,degree_max+1):
#     model = make_pipeline(PolynomialFeatures(degree, interaction_only=False), LassoCV(eps=lasso_eps,n_alphas=lasso_nalpha,max_iter=lasso_iter,
# normalize=True,cv=5))
#     model.fit(X_train,y_train)
#     test_pred = np.array(model.predict(X_test))
#     RMSE=np.sqrt(np.sum(np.square(test_pred-y_test)))
#     test_score = model.score(X_test,y_test)
#     print(degree, RMSE, test_score)

In [ ]:
# # find the top features that have a coefficient higher than L

# def find_best_features(coefficients, feature_names, lim=0.01):
#     strongest_features = {}
#     sorted_features_args = coefficients.argsort()[::-1]
#     for arg in sorted_features_args:
#         if coefficients[arg] > lim:
# #             strongest_features.append(feature_names[arg])
#             strongest_features[feature_names[arg]] = coefficients[arg]
#     return strongest_features

# find_best_features(
#     model.get_params()['lassocv'].coef_,
#     model.get_params()['polynomialfeatures'].get_feature_names(X.columns)
# )

In [ ]:
# from mpl_toolkits import mplot3d
# # gold prices

# # test_df.index
# # test_df.columns
# # test_df[['1w_return_US /Dollar', '1y_vol_US Dollar']]

# fig = plt.figure(figsize=(20, 20))
# ax = plt.axes(projection='3d')
# ax.view_init(10, 45)

# # Data for a three-dimensional line
# # zline = test_df.index
# # xline = test_df['1w_return_US Dollar']
# # yline = test_df['1y_vol_US Dollar']**2
# # ax.plot3D(xline, yline, zline, 'gray')

# # Data for three-dimensional scattered points
# zdata = test_df.index
# xdata = test_df['1w_return_US Dollar']
# ydata = test_df['1y_vol_US Dollar']**2

# ax.set_xlabel('USD Return')
# ax.set_ylabel('USD Annual Volatility')
# ax.set_zlabel('Date')

# ax.scatter3D(xdata, ydata, zdata, c=zdata, cmap='Greens')

In [ ]:
# test_df
# X = test_df[['1y_vol_US Dollar', '1y_vol_Japanese Yen']]
# y_currency = 'US Dollar'
# y_value = '1w_return'
# y_string = y_value + '_' + y_currency
# y = test_df[y_string]
# y
# X

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LassoCV
# from sklearn.cross_validation import train_test_split
# from sklearn.pipeline import make_pipeline

# lasso_eps = 0.0001
# lasso_nalpha=20
# lasso_iter=1000

# # Min and max degree of polynomials features to consider
# degree_min = 2
# degree_max = 2

# # Test/train split
# X_train, X_test, y_train, y_test = train_test_split(X, y)

# # Make a pipeline model with polynomial transformation and LASSO regression with cross-validation, run it for increasing degree of polynomial (complexity of the model)
# for degree in range(degree_min,degree_max+1):
#     model = make_pipeline(PolynomialFeatures(degree, interaction_only=False), LassoCV(eps=lasso_eps,n_alphas=lasso_nalpha,max_iter=lasso_iter,
# normalize=True,cv=5))
#     model.fit(X_train,y_train)
#     test_pred = np.array(model.predict(X_test))
#     RMSE=np.sqrt(np.sum(np.square(test_pred-y_test)))
#     test_score = model.score(X_test,y_test)
#     print(degree, RMSE, test_score)